<a href="https://colab.research.google.com/github/popescuaaa/playground/blob/master/Gans_and_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Create a simple neural network to classify iris plants

## Setup

In [106]:
import torch
import yaml
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np

## Custom dataset
## Must povide data + labels
General structure for data:
[<useless id>, f, f, f, f, l] f = fature, l = label

In [107]:
LABELS = ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']

In [108]:
class IrisDataset(Dataset):
    def __init__(self):
        self.raw_data = np.loadtxt('./Iris.csv', delimiter=',', dtype=np.str)
        self.len = self.raw_data.shape[0] - 1
        self.relevant_data = [[float(item[1]), float(item[2]), float(item[3]), float(item[4])] \
                     for item in self.raw_data[1:len(self.raw_data)]]

        # Processing data
        self.labels = [ [float(LABELS.index(item[5]))] for item in self.raw_data[1:len(self.raw_data)] ]

        self.data = [(None, None)] * len(self.relevant_data)

        for idx in range(len(self.relevant_data)):
          self.data[idx] = ( torch.Tensor(self.relevant_data[idx]), torch.Tensor(self.labels[idx]))

    def get_features_list(self):
      return self.raw_data[0][1:len(self.raw_data[0]) - 1]

    def __getitem__(self, index):
        data, label = self.data[index] # (data, label)
        return data, label

    def __len__(self):
        return self.len

In [109]:
ds = IrisDataset()

## Dataloader
Dataset + sampler

In [144]:
 # load yaml data
with open('simple_network.yaml', 'r') as f:
    config = yaml.load(f)

train_ds, test_ds = IrisDataset(), IrisDataset()

train_loader = DataLoader(dataset=train_ds, batch_size=int(config['batch_size']), shuffle=True)
test_loader = DataLoader(dataset=test_ds, batch_size=int(config['batch_size']), shuffle=True)


## NN

In [156]:
class IrisNet(nn.Module):
    def __init__(self, cfg):
        super(IrisNet, self).__init__()
        self.input_size = cfg['input_size']
        self.hidden_layer_1 = cfg['hidden_layer_1']
        self.hidden_layer_2 = cfg['hidden_layer_2']
        self.num_classes = cfg['num_classes']

        # define network structure
        self.fc1 = nn.Linear(self.input_size, self.hidden_layer_1)
        self.activation1 = nn.ReLU()

        self.fc2 = nn.Linear(self.hidden_layer_1, self.hidden_layer_2)
        self.activation2 = nn.ReLU()

        self.fc3 = nn.Linear(self.hidden_layer_2, self.num_classes)
        self.activation3 = nn.ReLU()

    def forward(self, batch):
        out = self.fc1(batch)
        out = self.activation1(out)
        out = self.fc2(out)
        out = self.activation2(out)
        out = self.fc3(out)
        out = self.activation3(out)
        return out



## Criterion and Optimizer

In [157]:
iris_nn = IrisNet(config)
criterion = nn.MSELoss()
optimizer = optim.SGD(iris_nn.parameters(), lr=0.001, momentum=0.9)

## Training

In [158]:

for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = iris_nn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if epoch % 5 == 0:
          print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([2, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,     1] loss: 0.000
[1,     2] loss: 0.001
[1,     3] loss: 0.001
[1,     4] loss: 0.001
[1,     5] loss: 0.000
[1,     6] loss: 0.000
[1,     7] loss: 0.001
[1,     8] loss: 0.000
[1,     9] loss: 0.001
[1,    10] loss: 0.000
[1,    11] loss: 0.000
[1,    12] loss: 0.000
[1,    13] loss: 0.000
[1,    14] loss: 0.000
[1,    15] loss: 0.000
[1,    16] loss: 0.000
[1,    17] loss: 0.000
[1,    18] loss: 0.000
[1,    19] loss: 0.000
[1,    20] loss: 0.000
[1,    21] loss: 0.000
[1,    22] loss: 0.000
[1,    23] loss: 0.000
[1,    24] loss: 0.000
[1,    25] loss: 0.000
[1,    26] loss: 0.000
[1,    27] loss: 0.000
[1,    28] loss: 0.000
[1,    29] loss: 0.000
[1,    30] loss: 0.000
[1,    31] loss: 0.000
[1,    32] loss: 0.000
[1,    33] loss: 0.000
[1,    34] loss: 0.000
[1,    35] loss: 0.000
[1,    36] loss: 0.000
[1,    37] loss: 0.000
[1,    38] loss: 0.000
[6,     1] loss: 0.000
[6,     2] loss: 0.000
[6,     3] loss: 0.000
[6,     4] loss: 0.000
[6,     5] loss: 0.001
[6,     6] 

In [159]:
PATH = './iris_net.pth'
torch.save(iris_nn.state_dict(), PATH)

In [160]:
iris_nn_test = IrisNet(config)
iris_nn_test.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [162]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        iris, labels = data
        outputs = iris_nn_test(iris)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on test iris data: %d %%' % (
    100 * correct / total))


Accuracy of the network on test iris data: 178 %


## Very Simple GAN to generate new iris data

In [221]:
# The discriminator will follow the ideea in the iris net above
# If the problem from above persits basically the Discriminator
# will be very good and will not let the Generator to make better samples

training_configuration = {
    'batch_size': 4,
    'lr': 3e-4,
    'z_size': 4,
    'num_epochs': 100
}

discriminator_in_features = {
    'input_size': 4, # the number of features
    'hidden_layer_1': 10,
    'hidden_layer_2': 10,
    'output_size': 3 # num_classes
}

generator_in_features = {
    'input_size': 4, # z_dim => noise
    'hidden_layer_1': 10,
    'hidden_layer_2': 10,
    'output_size': 4 # this will generate lists of 4 params that will be 
                    # interpreted as an iris configuration
}

In [222]:
class Generator(nn.Module):
  def __init__(self, in_features):
    super().__init__()
    self.params = in_features
    __module_list = [
        nn.Linear(self.params['input_size'], self.params['hidden_layer_1']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_1'], self.params['hidden_layer_2']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_2'], self.params['output_size']),
        nn.Tanh()
    ]

    self.__net = nn.Sequential(*__module_list)

  def forward(self, x):
     return self.__net(x)

In [223]:
class Discriminator(nn.Module):
  def __init__(self, in_features):
    super().__init__()
    self.params = in_features
    __module_list = [
        nn.Linear(self.params['input_size'], self.params['hidden_layer_1']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_1'], self.params['hidden_layer_2']),
        nn.ReLU(),
        nn.Linear(self.params['hidden_layer_2'], self.params['output_size']),
        nn.Sigmoid()
    ]

    self.__net = nn.Sequential(*__module_list)

  def forward(self, x):
     return self.__net(x)


## Training setup

In [224]:
D = Discriminator(discriminator_in_features)
G = Generator(generator_in_features)
optimizer_D = optim.Adam(D.parameters(), lr=training_configuration['lr'])
optimizer_G = optim.Adam(G.parameters(), lr=training_configuration['lr'])
criterion = nn.BCELoss()
fixed_noise = torch.randn((training_configuration['batch_size'], training_configuration['z_size']))

## Load data

In [225]:
ds = IrisDataset()
loader = DataLoader(dataset=ds, batch_size=int(training_configuration['batch_size']), shuffle=True)

## Trainig loop

In [226]:
for epoch in range(training_configuration['num_epochs']):
    for batch_idx, (real, _) in enumerate(loader):
        batch_size = real.shape[0]

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        noise = torch.randn(training_configuration['batch_size'], training_configuration['z_size'])

        fake = G(noise)

        D_real = D.forward(real)

        lossD_real = criterion(D_real, torch.ones_like(D_real))

        D_fake = D.forward(fake)

        lossD_fake = criterion(D_fake, torch.zeros_like(D_fake))

        lossD = (lossD_real + lossD_fake) / 2

        D.zero_grad()

        lossD.backward(retain_graph=True)

        optimizer_D.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        # where the second option of maximizing doesn't suffer from
        # saturating gradients

        # G(z) = fake

        output = D.forward(fake)

        lossG = criterion(output, torch.ones_like(output))

        G.zero_grad()

        lossG.backward()

        optimizer_G.step()

        if batch_idx == 0:
            print(
                f"Epoch [{epoch}/{training_configuration['num_epochs']}] Batch {batch_idx}/{len(loader)} \
                      Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
            )

            with torch.no_grad():
                fake = G(fixed_noise)
                data = real
                print('Fake: {}  Real: {}'.format(fake, real))

Epoch [0/100] Batch 0/38                       Loss D: 0.7264, loss G: 0.7435
Fake: tensor([[-0.1334, -0.2132,  0.3222,  0.1404],
        [-0.2095, -0.0364,  0.2580,  0.1810],
        [-0.1135, -0.2460,  0.3273,  0.1192],
        [-0.1100, -0.1767,  0.3087,  0.1576]])  Real: tensor([[6.6000, 2.9000, 4.6000, 1.3000],
        [4.3000, 3.0000, 1.1000, 0.1000],
        [5.1000, 3.8000, 1.6000, 0.2000],
        [5.1000, 3.8000, 1.5000, 0.3000]])
Epoch [1/100] Batch 0/38                       Loss D: 0.7084, loss G: 0.7290
Fake: tensor([[-0.1302, -0.1389,  0.2587,  0.2076],
        [-0.2035,  0.0157,  0.2120,  0.2258],
        [-0.1145, -0.1609,  0.2520,  0.1999],
        [-0.1102, -0.0875,  0.2313,  0.2393]])  Real: tensor([[6.1000, 2.6000, 5.6000, 1.4000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [5.5000, 2.3000, 4.0000, 1.3000],
        [5.5000, 2.5000, 4.0000, 1.3000]])
Epoch [2/100] Batch 0/38                       Loss D: 0.6816, loss G: 0.7231
Fake: tensor([[-0.0803, -0.0559,